<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Mask_R_CNN_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/ftaubner/semantic_features_detection.git

Cloning into 'semantic_features_detection'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1420 (delta 13), reused 18 (delta 8), pack-reused 1392
Receiving objects: 100% (1420/1420), 114.23 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (865/865), done.


In [0]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q" -O kitti.zip

--2020-04-14 10:40:11--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://xwp44a.am.files.1drv.com/y4m6Fg6gPg5LjCObuIXylD9aIbelbeAXOfKuo1U5nnsBkX4_Povrw9Kr3WjEsNfYjt4uxnZjzDIsBpTnMdl_rHyhULyL8g8DiGSHr6KDb3DHtFlcqEPr08QWpU40prpMFJF0h8e0gvYbG7d4-LbiiDscJ8AJMorf_bLNptY-Er_yp0l-wyLrOL7DM6X8k8bFpxGL3Lb9EBWNP4k9WCBZBSvkw/kitti_dataset.zip?download&psid=1 [following]
--2020-04-14 10:40:12--  https://xwp44a.am.files.1drv.com/y4m6Fg6gPg5LjCObuIXylD9aIbelbeAXOfKuo1U5nnsBkX4_Povrw9Kr3WjEsNfYjt4uxnZjzDIsBpTnMdl_rHyhULyL8g8DiGSHr6KDb3DHtFlcqEPr08QWpU40prpMFJF0h8e0gvYbG7d4-LbiiDscJ8AJMorf_bLNptY-Er_yp0l-wyLrOL7DM6X8k8bFpxGL3Lb9EBWNP4k9WCBZBSvkw/kitti_dataset.zip?download&psid=1
Resolving xwp44a.am.files.1drv.com (xwp44a.am.fi

In [0]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100479&authkey=AJdjSPnfz5shCvk" -O mapillary_47.h5

--2020-04-14 10:41:11--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100479&authkey=AJdjSPnfz5shCvk
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://qyvpsa.am.files.1drv.com/y4mk-0QeIaZXflhY0EjwLgxoJMnDTysbuC-X9ag_bm02gfrrobyCar1l9n32rOg-4mBIe4pUpP1ZipdanvyK23SqGQ4jEeBotggdM386Ysv5uBTP4-cwyL6vWorxJ2Y1dSK_HL1gkPz9uFpshmKRwC35rwHlfsniTQ75pVCJ6GYRvfWQdEQcwTcwhaXwv0t128WJRLjlBDEoHe9vpKpPGeJog/mask_rcnn_mapvistas_0047.h5?download&psid=1 [following]
--2020-04-14 10:41:11--  https://qyvpsa.am.files.1drv.com/y4mk-0QeIaZXflhY0EjwLgxoJMnDTysbuC-X9ag_bm02gfrrobyCar1l9n32rOg-4mBIe4pUpP1ZipdanvyK23SqGQ4jEeBotggdM386Ysv5uBTP4-cwyL6vWorxJ2Y1dSK_HL1gkPz9uFpshmKRwC35rwHlfsniTQ75pVCJ6GYRvfWQdEQcwTcwhaXwv0t128WJRLjlBDEoHe9vpKpPGeJog/mask_rcnn_mapvistas_0047.h5?download&psid=1
Resolving qyvpsa.am.files.1d

In [0]:
!unzip -qq /content/kitti.zip -d /content

In [0]:
!pip install imgaug
!pip install Cython
!pip install pycocotools
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=f34f80f07a7e97ac30917955a49f6fe3c006dc7cddd17f6af64f8eee3a5476f6
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
%tensorflow_version 1.x
%matplotlib inline
import os
import sys
os.chdir('/content/semantic_features_detection/Notebook')
ROOT_DIR = os.path.abspath('../')
sys.path.append(os.path.join(ROOT_DIR, 'samples'))
import inference

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
inf_model = inference.Inference(os.path.join(ROOT_DIR, 'mapillary_47.h5'))
features, masks = inf_model.predict('/content/dataset/sequences/04/image_2/000001.png')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
/content/dataset/sequences/04/image_2/000001.png



In [0]:
print(features)

{'results': [{'image_id': '000001', 'classes': [15, 10, 10, 15, 10, 10, 1, 1, 10, 10, 10, 15, 9, 10, 10, 10, 10, 10, 10], 'boxes': [[119, 806, 143, 826], [15, 810, 213, 820], [52, 78, 237, 93], [224, 1139, 275, 1165], [16, 707, 175, 721], [109, 481, 185, 486], [171, 220, 198, 239], [1, 45, 53, 108], [16, 42, 232, 54], [41, 393, 200, 416], [141, 547, 170, 561], [198, 906, 221, 918], [1, 967, 23, 986], [103, 1150, 273, 1166], [73, 223, 182, 236], [82, 390, 201, 402], [137, 540, 177, 555], [107, 476, 187, 484], [145, 553, 173, 569]]}]}


In [0]:
import numpy as np
def masks2instance_im(masks):
    instance_im = np.zeros(shape=[masks.shape[0], masks.shape[1]])
    for i in range(masks.shape[-1]):
        instance_im[masks[...,i]] = (i + 1)
    return instance_im

In [0]:
import glob
import cv2
IMAGE_DIR_LEFT = '/content/dataset/sequences/04/image_2'
IMAGE_DIR_RIGHT = '/content/dataset/sequences/04/image_3'
SAVE_DIR = '/content/Results/Instances'

features = {}
features['results'] = []
image_paths = glob.iglob(os.path.join(IMAGE_DIR_LEFT, '*.*'))
for image_path in image_paths:
    feature, masks = inf_model.predict(image_path)
    features['results'].append(feature)
    # save instance images
    instance_im =  masks2instance_im(masks)
    image_id=os.path.split(image_path)[1][0:-4]
    cv2.imwrite(os.path.join(SAVE_DIR, 'L' + image_id + '.png'), instance_im)


/content/dataset/sequences/04/image_2/000227.png
/content/dataset/sequences/04/image_2/000174.png
/content/dataset/sequences/04/image_2/000047.png
/content/dataset/sequences/04/image_2/000078.png
/content/dataset/sequences/04/image_2/000236.png
/content/dataset/sequences/04/image_2/000042.png
/content/dataset/sequences/04/image_2/000191.png
/content/dataset/sequences/04/image_2/000026.png
/content/dataset/sequences/04/image_2/000040.png
/content/dataset/sequences/04/image_2/000214.png
/content/dataset/sequences/04/image_2/000188.png


KeyboardInterrupt: ignored

In [0]:
import json
FEATURE_FILE = '/content/results.json'
with open(FEATURE_FILE, 'w') as outfile:
    json.dump(features, outfile)